# Imports

In [11]:
import string

In [12]:
from pathlib import Path
import sys

In [13]:
cwd = Path.cwd()

In [14]:
stanford_dir = cwd.parent / 'XCS224N-A1-master' / 'src'

In [15]:
sys.path.append(str(stanford_dir))

In [16]:
import sys
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from utils import *
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

[nltk_data] Downloading package reuters to /Users/mukti/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


# Solutions

In [143]:
def distinct_words(corpus):
    """ Determine a list of distinct words for the corpus.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
()            corpus_words (list of strings): list of distinct words across the corpus, sorted (using python 'sorted' function)
            num_corpus_words (integer): number of distinct words across the corpus
    """
    corpus_words = []
    num_corpus_words = 0

    # ### START CODE HERE ###
    corpus_words = sorted(list({item for document in corpus for item in document if item.isalpha() or item in ['<START>','<END>']}))
    num_corpus_words = len(corpus_words)
    # ### END CODE HERE ###

    return corpus_words, num_corpus_words

In [144]:
# example
c1 = '<START> all that glitters is not gold . <END>'
c2 = '<START> all is well that ends well . <END>'

corpus = [c1.split(), c2.split()]

In [145]:
corpus

[['<START>', 'all', 'that', 'glitters', 'is', 'not', 'gold', '.', '<END>'],
 ['<START>', 'all', 'is', 'well', 'that', 'ends', 'well', '.', '<END>']]

In [146]:
corpus_words, num_corpus_words = distinct_words(corpus)

In [147]:
corpus_words

['<END>',
 '<START>',
 'all',
 'ends',
 'glitters',
 'gold',
 'is',
 'not',
 'that',
 'well']

In [148]:
num_corpus_words

10

In [24]:
reuters_corpus = read_corpus()[100]

In [25]:
reuters_corpus

['<START>',
 'texaco',
 '&',
 'lt',
 ';',
 'txc',
 '>',
 'canada',
 'to',
 'raise',
 'crude',
 'oil',
 'postings',
 'texaco',
 'inc',
 "'",
 's',
 'texaco',
 'canada',
 'said',
 'it',
 'will',
 'raise',
 'postings',
 'for',
 'its',
 'edmonton',
 '/',
 'swann',
 'hills',
 'crude',
 'by',
 '24',
 'canadian',
 'cts',
 'a',
 'barrel',
 ',',
 'effective',
 'june',
 '20',
 '.',
 'the',
 'company',
 'said',
 'the',
 'new',
 'posting',
 'for',
 'edmonton',
 '/',
 'swann',
 'hills',
 'will',
 'be',
 '25',
 '.',
 '60',
 'dlrs',
 'a',
 'barrel',
 '.',
 'the',
 'price',
 'hike',
 'follows',
 'a',
 'round',
 'of',
 'crude',
 'oil',
 'price',
 'increases',
 'started',
 'late',
 'june',
 '17',
 'by',
 'sun',
 'co',
 '.',
 'the',
 'other',
 'major',
 'canadian',
 'crude',
 'suppliers',
 'raised',
 'prices',
 'june',
 '18',
 '.',
 '<END>']

In [149]:
def compute_co_occurrence_matrix(corpus, window_size=4):
    """ Compute co-occurrence matrix for the given corpus and window_size (default of 4).

        Note: Each word in a document should be at the center of a window. Words near edges will have a smaller
              number of co-occurring words.

              For example, if we take the document "START All that glitters is not gold END" with window size of 4,
              "All" will co-occur with "START", "that", "glitters", "is", and "not".

        Params:
            corpus (list of list of strings): corpus of documents
            window_size (int): size of context window
        Return:
            M (numpy matrix of shape (number of unique words in the corpus , number of unique words in the corpus)):
                Co-occurrence matrix of word counts.
                The ordering of the words in the rows/columns should be the same as the ordering of the words given by the distinct_words function.
            word2Ind (dict): dictionary that maps word to index (i.e. row/column number) for matrix M.
    """
    words, num_words = distinct_words(corpus)
    M = None
    word2Ind = {}

    # ### START CODE HERE ###
    # initialize a matrix of size num_words
    M = np.zeros((num_words,num_words))
    
    # create a dictionary that takes the sorted words list and then assigns an index to each
    # This dict will help in mapping the word pairs to add values in the M matrix
    word2Ind = dict((word,n) for n, word in enumerate(words))
    
    # create a clean corpus structure that eliminates characters but keeps the list of lists
    cleaned_corpus = [[item for item in document if item.isalpha() or item in ['<START>','<END>']] for document in corpus]
    
    # cleaned_corpus is a list of lists. Each list is a document.
    # Each list has strings
    for document in cleaned_corpus:
        # loop over each word that would be a center word
        for center_i in range(len(document)):
            # this will only select indices that span the window around the center word and within range
            # context_m is the index of a word present around the center word in a window of size window_size
            for context_m in filter(lambda x: 0 <= x <= len(document)-1, 
                                map(lambda x: x + center_i, range(-window_size,window_size+1))):
                
                # get the center word and context word
                center_word = document[center_i]
                context_word = document[context_m]
                
                # access these words from the dictionary
                # Allot the row index of M to be for center words and column index for the context word
                if center_word in word2Ind and context_word in word2Ind:
                    M[word2Ind[center_word],word2Ind[context_word]] += 1
                    
    # ### END CODE HERE ###

    return M, word2Ind

In [103]:
list(map(lambda x: x + 2 ,range(-4,5)))

[-2, -1, 0, 1, 2, 3, 4, 5, 6]

In [105]:
for n in map(lambda x: x + 2 ,range(-4,5)):
    print(n)

-2
-1
0
1
2
3
4
5
6


In [122]:
word2Ind

{'<END>': 0,
 '<START>': 1,
 'all': 2,
 'ends': 3,
 'glitters': 4,
 'gold': 5,
 'is': 6,
 'not': 7,
 'that': 8,
 'well': 9}

In [125]:
cleaned_corpus

[['<START>', 'all', 'that', 'glitters', 'is', 'not', 'gold', '<END>'],
 ['<START>', 'all', 'is', 'well', 'that', 'ends', 'well', '<END>']]

In [137]:
x = np.zeros((3,3))

In [138]:
x

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [139]:
test_dict = {'cest':0,
             'un': 1,
             'dimanche':2}

In [140]:
x[test_dict['cest'],test_dict['un']] += 1

In [142]:
x

array([[0., 1., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])